In [34]:
import requests as req
from bs4 import BeautifulSoup    
import pandas as pd
import numpy as np

In [2]:
raw_page = req.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [3]:
#method which will take HTML table as input and return pandas dataframe
def parse_table(table,columns_headers):
    n_columns = 0
    n_rows=0
    for row in table.find_all('tr'):
        column_data = row.find_all('td')
        if len(column_data) > 0:
            n_rows+=1
            if n_columns == 0:
                n_columns = len(column_data)
    df = pd.DataFrame(columns = columns_headers,index= range(0,n_rows))
    row_marker = 0
    for row in table.find_all('tr'):
        column_marker = 0
        columns = row.find_all('td')
        for column in columns:
            df.iat[row_marker,column_marker] = column.get_text()
            column_marker += 1
        if len(columns) > 0:
            row_marker += 1
    return df

In [4]:
soup = BeautifulSoup(raw_page, 'lxml')
dataFrm = parse_table(soup.find_all('table')[0],['Postcode','Borough','Neighbourhood'])

In [5]:
dataFrm['Neighbourhood'] = dataFrm.Neighbourhood.str.replace(r'\n', '')

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [6]:
df = dataFrm[dataFrm['Borough']!='Not assigned']

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [7]:
 df = df.groupby(['Postcode','Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)])

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [8]:
df.reset_index(level=['Postcode', 'Borough'], inplace=True)

In [9]:
def correctNeighbourhood(row):
    ans = ''
    if(row['Neighbourhood']=='Not assigned'):
        ans = row['Borough']
    else:
        ans = row['Neighbourhood']
    row['Neighbourhood']=ans    
    return row
df = df.apply(lambda row : correctNeighbourhood(row), axis=1)

In [10]:
df.shape

(103, 3)

Instead of geocoder package, using a csv file that has the geographical coordinates of each postal code

In [11]:
import io
csv_url="http://cocl.us/Geospatial_data"
csv_data=req.get(csv_url).content
csv=pd.read_csv(io.StringIO(csv_data.decode('utf-8')))

In [12]:
finalDF = df.join(csv)

In [13]:
# grabbed relevant imports required for visualization and cluster analysis
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



In [16]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

# All requested packages already installed.



In [14]:
final_data = finalDF[finalDF['Borough'].str.contains('Toronto')].reset_index(drop=True)
final_data.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",M4L,43.668999,-79.315572
3,M4M,East Toronto,Studio District,M4M,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790


In [46]:
finalDF[finalDF['Borough'].str.contains('ronto')].reset_index(drop=True)

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",M4L,43.668999,-79.315572
3,M4M,East Toronto,Studio District,M4M,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,M4P,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,M4R,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,M4S,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",M4T,43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",M4V,43.686412,-79.400049


In [17]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Manhattan are 43.653963, -79.387207.


In [18]:
CLIENT_ID = 'RFVDSYOWWO2LFEB0Q4T3EYEPU2HFGSIYE35LMXNXIZT0H302' # your Foursquare ID
CLIENT_SECRET = 'E5BWKVHOKS4QCACDRQLAFVWRHUB0T3U5NGZXFF0MRB2HF13T' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RFVDSYOWWO2LFEB0Q4T3EYEPU2HFGSIYE35LMXNXIZT0H302
CLIENT_SECRET:E5BWKVHOKS4QCACDRQLAFVWRHUB0T3U5NGZXFF0MRB2HF13T


In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = req.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
# type your answer here
toronto_venues = getNearbyVenues(names=final_data['Neighbourhood'],latitudes=final_data['Latitude'],longitudes=final_data['Longitude'])

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Bar,Asian Restaurant,Restaurant,Hotel,Clothing Store
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Café,Cheese Shop,Seafood Restaurant,Farmers Market,Steakhouse,Bakery,Beer Bar
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Grocery Store,Falafel Restaurant,Performing Arts Venue,Bakery,Pet Store,Convenience Store,Restaurant
3,Business reply mail Processing Centre969 Eastern,Light Rail Station,Farmers Market,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Boat or Ferry,Sculpture Garden
5,"Cabbagetown, St. James Town",Restaurant,Coffee Shop,Pub,Chinese Restaurant,Pizza Place,Bakery,Italian Restaurant,Indian Restaurant,Café,Farmers Market
6,Central Bay Street,Coffee Shop,Italian Restaurant,Japanese Restaurant,Chinese Restaurant,Café,Bar,Bubble Tea Shop,Ice Cream Shop,Burger Joint,Thai Restaurant
7,"Chinatown, Grange Park, Kensington Market",Café,Bar,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Bakery,Mexican Restaurant,Coffee Shop,Dumpling Restaurant,Chinese Restaurant,Gaming Cafe
8,Christie,Grocery Store,Café,Park,Convenience Store,Nightclub,Baby Store,Restaurant,Athletics & Sports,Diner,Italian Restaurant
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Burger Joint,Restaurant,Mediterranean Restaurant,Pub,Café,Fast Food Restaurant


In [37]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [42]:
toronto_merged = final_data
toronto_merged.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)
# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031,0,Coffee Shop,Pub,Women's Store,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188,0,Greek Restaurant,Ice Cream Shop,Coffee Shop,Italian Restaurant,Bookstore,Yoga Studio,Furniture / Home Store,Bakery,Spa,Juice Bar
2,M4L,East Toronto,"The Beaches West, India Bazaar",M4L,43.668999,-79.315572,0,Park,Sushi Restaurant,Brewery,Sandwich Place,Burger Joint,Burrito Place,Pub,Pizza Place,Coffee Shop,Pet Store
3,M4M,East Toronto,Studio District,M4M,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Gastropub,Yoga Studio,Chinese Restaurant,Fish Market,Latin American Restaurant
4,M4N,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790,0,Bus Line,Lake,Dim Sum Restaurant,Park,Swim School,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Women's Store


In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters